In [1]:
from rhp_plan import plan
from pathlib import Path
import openpyxl
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from copy import deepcopy


#Excel information extraction

xlsx_file = Path('nrio_sut_181108.xlsx')
main_sheet = openpyxl.load_workbook(xlsx_file)['2008']
xlsx_file = Path('posternas_namn.xlsx')
shorthand_sheet = openpyxl.load_workbook(xlsx_file)['SUP10']

#Division with zero-exception
def zdivide(x, y):
    return np.divide(x, y, out=np.zeros_like(x), where=y!=0)

#List constructor
def list_maker(x, y):
    return ([x]*y)    

#Reshapers
def reshape_col(x):
    return(x.reshape([-1,1]).reshape([1,-1]))
def reshape_row(x):
    return(x.reshape([1,-1]).reshape([-1,1]))

# Generates a 2D list of values from an xlsx file, reading left to right, top to bottom.
class Sheet:
    def __init__(self, sheet, left: int, top: int, right: int, bottom: int):
        result_sheet = []
        coordinates = []
        for row in sheet.iter_rows(min_row=left, min_col=top, max_row=right, max_col=bottom):
            data = []
            pos = []
            for cell in row:
                data.append(cell.value)
                pos.append(cell)

            result_sheet.append(data)
            coordinates.append(pos)

        self.result = result_sheet
        self.coordinates = coordinates

    @property
    def np_array(self):
        return np.array(self.result, dtype=np.float64)

#Input values
time_steps = 3
planning_horizon = 3
step_horizon = time_steps + planning_horizon
carbon_dioxide_emissions = Sheet(main_sheet, 69, 3, 87, 61).np_array[-1]
supply = np.matrix(Sheet(main_sheet, 159, 3, 217, 61).np_array)
use_domestic = np.matrix(Sheet(main_sheet, 4, 3, 62, 61).np_array)
use_imported = np.matrix(Sheet(main_sheet, 94, 3, 152, 61).np_array)
target_output = np.matrix(reshape_row(np.sum(Sheet(main_sheet, 4, 68, 62, 74).np_array + Sheet(main_sheet, 94, 68, 152, 74).np_array, axis=1)))
export_vector = np.matrix(Sheet(main_sheet, 4, 75, 62, 75).np_array + Sheet(main_sheet, 94, 75, 152, 75).np_array)
export_prices = np.matrix(np.array([[1] for i in range((export_vector).shape[0]+1)]))
import_prices = np.matrix(np.array([[1] for i in range((export_vector).shape[0]+1)]))
depreciation = np.matrix(np.array(np.eye(N=supply.shape[0]+1, M=supply.shape[0]+1, k=0)))
primary_resource = np.matrix(np.array([[1] for i in range(planning_horizon * Sheet(main_sheet, 159, 3, 217, 61).np_array.shape[1])]))
worked_hours = np.matrix(Sheet(main_sheet, 69, 3, 87, 61).np_array[-2])

#List construction
supply_list = list_maker(np.row_stack((supply, carbon_dioxide_emissions)), step_horizon)
use_domestic_list = list_maker(np.row_stack((use_domestic, np.array([0]*use_domestic.shape[1]))), step_horizon)
use_imported_list = list_maker(np.row_stack((use_imported, np.array([0]*use_imported.shape[1]))), step_horizon)
target_output_list = list_maker(np.row_stack((target_output*(1/12), np.array([[0]]))), step_horizon)
export_vector_list = list_maker(np.row_stack((export_vector, np.array([0]))), step_horizon)
export_prices_list = list_maker(export_prices, step_horizon)
import_prices_list = list_maker(import_prices, step_horizon)
depreciation_list = list_maker(depreciation, (step_horizon + 1))
primary_resource_list = list_maker(primary_resource, step_horizon)

print(supply_list[0].shape)
print(use_domestic_list[0].shape)
print(use_imported_list[0].shape)
print(target_output_list[0].shape)
print(export_vector_list[0].shape)
print(export_prices_list[0].shape)
print(import_prices_list[0].shape)
print(depreciation_list[0].shape)
print(primary_resource_list[0].shape)

print(type(supply_list[0]))
print(type(use_domestic_list[0]))
print(type(use_imported_list[0]))
print(type(target_output_list[0]))
print(type(export_vector_list[0]))
print(type(export_prices_list[0]))
print(type(import_prices_list[0]))
print(type(depreciation_list[0]))
print(type(primary_resource_list[0]))



#Notation
shorthand = [[x[0][4:] if x[0].startswith('CPA_') else x[0], x[1]] for x in Sheet(shorthand_sheet, 7, 2, 65, 3).result]
sector = Sheet(main_sheet, 3, 3, 3, 61)
[print(f'{short}: {desc}') for short, desc in shorthand]
sector_name_input = deepcopy(sector.result[0])
sector_with_all_outputs_input = deepcopy(sector_name_input)
sector_with_all_outputs_input.append('CO2')
sector_with_all_outputs_and_EXP = deepcopy(sector_with_all_outputs_input)
sector_with_all_outputs_and_EXP.append('EXP')

#Plan
plan_outcome = plan(time_steps=time_steps, planning_horizon=planning_horizon, primary_resource_list=primary_resource_list, 
                    supply_list=supply_list, use_domestic_list=use_domestic_list, use_imported_list=use_imported_list,
                    depreciation_list=depreciation_list, target_output_list=target_output_list, export_vector_list=export_vector_list, 
                    export_prices_list=export_prices_list, import_prices_list=import_prices_list)

# plan details
result_list = plan_outcome[0]
lagrange_list = plan_outcome[1]
slack_list = plan_outcome[2]
for T in range(time_steps):
    x = np.array_split(result_list[T], planning_horizon)
    l = np.array_split(lagrange_list[T], planning_horizon)
    s = np.array_split(slack_list[T], planning_horizon)

plt.style.use('_mpl-gallery')
fig = plt.figure(figsize=(30, 180))
fig.suptitle('Results', fontsize=32)
gs = gridspec.GridSpec(5 * time_steps * planning_horizon, 1)

labels = ['overshoot_target_output_quotient',
            'Worked hours (10K)',
            'Worked hours (10K) percentage',
            'Lagrange multiplier']

for i in range(time_steps):
    ax = fig.add_subplot(gs[i * 5 + 1, 0])
    ax.set_xlabel('Product', fontsize=14)
    ax.set_ylabel(labels[0], fontsize=14)
    ax.set_xticks(range(supply_list[i].shape[0]), sector_with_all_outputs_input)
    for j in range(planning_horizon):
        l = np.array_split(slack_list[i], planning_horizon)
        ax.plot(range(s[j].shape[0]), s[j])

    ax = fig.add_subplot(gs[i * 5 + 2, 0])
    ax.set_xlabel('Product', fontsize=14)
    ax.set_ylabel(labels[1], fontsize=14)
    ax.set_xticks(range(supply_list[i].shape[1]), sector_name_input)
    for j in range(planning_horizon):
        x = np.array_split(result_list[i], planning_horizon)
        ax.plot(range(x[j].shape[0]), x[j])

    ax = fig.add_subplot(gs[i * 5 + 3, 0])
    ax.set_xlabel('Product', fontsize=14)
    ax.set_ylabel(labels[2], fontsize=14)
    ax.set_xticks(range(supply_list[i].shape[1]), sector_name_input)
    for j in range(planning_horizon):
        x = np.array_split(result_list[i], planning_horizon)
        ax.plot(range(x[j].shape[0]), x[j] / worked_hours)

    ax = fig.add_subplot(gs[i * 5 + 4, 0])
    ax.set_xlabel('Product', fontsize=14)
    ax.set_ylabel(labels[3], fontsize=14)
    ax.set_xticks(range(len(sector_with_all_outputs_and_EXP)), sector_with_all_outputs_and_EXP)
    for j in range(planning_horizon):
        l = np.array_split(lagrange_list[i], planning_horizon)
        ax.plot(range(l[j].shape[0]), l[j])

plt.show()


C:\Users\loke_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


(60, 59)
(60, 59)
(60, 59)
(60, 1)
(60, 1)
(60, 1)
(60, 1)
(60, 60)
(177, 1)
<class 'numpy.matrix'>
<class 'numpy.matrix'>
<class 'numpy.matrix'>
<class 'numpy.matrix'>
<class 'numpy.matrix'>
<class 'numpy.matrix'>
<class 'numpy.matrix'>
<class 'numpy.matrix'>
<class 'numpy.matrix'>
A01: Products of agriculture, hunting and related services
A02: Products of forestry, logging and related services
A03: Fish and other fishing products; aquaculture products; support services to fishing
B: Mining and quarrying
C10T12: Food products, beverages and tobacco products
C13T15: Textiles, wearing apparel and leather products
C16: Wood and of products of wood and cork, except furniture; articles of straw and plaiting materials
C17: Paper and paper products
C18: Printing and recording services
C19: Coke and refined petroleum products 
C20-21: Chemicals and chemical products AND Basic pharmaceutical products and pharmaceutical preparations
C22: Rubber and plastics products
C23: Other non-metallic mine

TypeError: bad operand type for unary -: 'NoneType'